# Handling Exceptions

When a exception is not handled, Python aborts the program

## `raise`

This is Jupyter catching the exception and output them for us. Does not crash Jupyter bc Jupyter is handling

In [3]:
raise ValueError('custom exception')

ValueError: custom exception

Custom exception handling:

In [4]:
try:
    raise ValueError('custom message')
except ValueError as ex:
    print(ex)

custom message


In [5]:
try:
    raise ValueError('custom message', 'secondary message')
except ValueError as ex:
    print(repr(ex))

ValueError('custom message', 'secondary message')


In [8]:
def func1():
    raise ValueError('bad value')

try:
    func1()
except ValueError as ex:
    print('handling value erro: ', repr(ex))
except IndexError as ex:
    print('handling index error: ', repr(ex))

handling value erro:  ValueError('bad value')


In [9]:
def func1():
    raise IndexError('bad value')

try:
    func1()
except ValueError as ex:
    print('handling value erro: ', repr(ex))
except IndexError as ex:
    print('handling index error: ', repr(ex))

handling index error:  IndexError('bad value')


In [10]:
def func1():
    raise TypeError('bad value')

try:
    func1()
except ValueError as ex:
    print('handling value erro: ', repr(ex))
except IndexError as ex:
    print('handling index error: ', repr(ex))

TypeError: bad value

The order thar the excptions are handled is also important, we should start from the most specific then go general

In [13]:
try:
    raise ValueError('error')
except Exception as e:
    print('handling exception: ', repr(e))
except ValueError as e:
    print('handling value error: ', repr(e))

handling exception:  ValueError('error')


In [14]:
try:
    raise ValueError('error')
except ValueError as e:
    print('handling value error: ', repr(e))
except Exception as e:
    print('handling exception: ', repr(e))

handling value error:  ValueError('error')


In [15]:
try:
    raise ValueError('error')
except TypeError as e:
    print('handling value error: ', repr(e))
except Exception as e:
    print('handling exception: ', repr(e))

handling exception:  ValueError('error')


## `finally`

Guaranteed to run, no matter what

In [16]:
try:
    raise ValueError()
except ValueError:
    print('handled error')
    raise KeyError()
finally:
    print('running finally')

handled error
running finally


KeyError: 

## `else`

In [17]:
try:
    a = 10 
except ValueError:
    print('value erorr')
else:
    print('no exception')

no exception


In [18]:
try:
    raise ValueError() 
except ValueError:
    print('value erorr')
else:
    print('no exception')

value erorr


In [19]:
try:
    raise ValueError() 
else:
    print('no exception')

SyntaxError: expected 'except' or 'finally' block (741212836.py, line 3)

In [20]:
try:
    raise ValueError() 
except:
    print('exception....')
else:
    print('no exception')

exception....


In [22]:
try:
    raise ValueError() 
except:
    print('exception....')
    raise
else: # runs when no exception has occurred
    print('no exception')

exception....


ValueError: 

In [23]:
try:
    pass 
except ValueError:
    print('value error')
else:
    print('no exception')

no exception


In [ ]:
def func(a):
    if a < 10:
        return True 
    else: 
        return False 

In [ ]:
def func(a):
    if a < 10:
        return True 
    return False 

Not equivalent in exception handling

In [27]:
try:
    raise ValueError
except ValueError:
    print('value error')

print('no exception')

value error
no exception


In [6]:
import json

json_data = """{
    "Alex": {"age": 23},
    "Bryan": {"age": 23, "city": "London"},
    "Guido": {"age": "unknown"}
}"""

In [7]:
data = json.loads(json_data)

In [11]:
data

{'Alex': {'age': 23},
 'Bryan': {'age': 23, 'city': 'London'},
 'Guido': {'age': 'unknown'}}

In [9]:
class Person:
    __slots__ = 'name', '_age'

    def __init__(self, name):
        self.name = name 
        self._age = None 

    @property 
    def age(self):
        return self._age
    
    @age.setter
    def age(self, value):
        if isinstance(value, int) and value >= 0:
            self._age = value 
        else:
            raise ValueError('Invalid age')
        
    def __repr__(self):
        return f"Person(name={self.name}, age={self.age})"

In [15]:
persons = []

for name, attributes in data.items():
    person = Person(name=name)
    skip_person = False
    for attrib_name, attrib_value in attributes.items():
        try:
            setattr(person, attrib_name, attrib_value)
        except AttributeError:
            print(f'ignoring attribute:{name}.{attrib_name}={attrib_value})')
        except ValueError as ex:     
            print(f'Data for Person({name}) contains an invalid attribute value: {ex}')
            skip_person = False 
            break
    if not skip_person:
        persons.append(person)

ignoring attribute:Bryan.city=London)
Data for Person(Guido) contains an invalid attribute value: Invalid age


More elegant way using nested try's

In [16]:
persons = []

for name, attributes in data.items():
    try:
        person = Person(name=name)
        for attrib_name, attrib_value in attributes.items():
            try:
                setattr(person, attrib_name, attrib_value)
            except AttributeError:
                print(f'ignoring attribute:{name}.{attrib_name}={attrib_value})')
    except ValueError as ex:     
        print(f'Data for Person({name}) contains an invalid attribute value: {ex}')
    else:
        persons.append(person)

ignoring attribute:Bryan.city=London)
Data for Person(Guido) contains an invalid attribute value: Invalid age


In [18]:
persons

[Person(name=Alex, age=23), Person(name=Bryan, age=23)]

In [19]:
def convert_int(val):
    if not isinstance(val, int):
        raise TypeError()
    if val not in [0, 1]:
        raise ValueError('Integer values 0 or 1 only')
    return bool(val)

In [26]:
def convert_str(val):
    if not isinstance(val, str):
        raise TypeError()
    
    val = val.casefold() # case insensitive comparision
    if val in ['0', 'f' or 'false']:
        return False
    elif val in ['1', 't', 'true']:
        return True 
    else:
        raise ValueError('Inadimissible string values')

In [21]:
class ConversionError(Exception):
    pass 

Try something and then ask for forgiveness

In [28]:
def make_bool(val):
    try:
        try:
            b = convert_int(val)
        except TypeError:
            try:
                b = convert_str(val)
            except TypeError as te:
                raise ConversionError(f"Type inadimissible: {te}")
    except ValueError as ex:
        raise ConversionError(f'The value could not be converted to boolean: {ex}')
    else:
        return b

In [29]:
values = [True, 0, 'T', 'false', 10, 'ABC', 1.0]

for value in values:
    try:
        result = make_bool(val=value)
    except ConversionError as ex:
        result = str(ex)

    print(value, result)

True True
0 False
T True
false The value could not be converted to boolean: Inadimissible string values
10 The value could not be converted to boolean: Integer values 0 or 1 only
ABC The value could not be converted to boolean: Inadimissible string values
1.0 Type inadimissible: 


Ask permission then try something

In [32]:
def make_bool(val):
    if isinstance(val, int):
        if val in [0, 1]:
            return bool(val)
        else:
            raise ConversionError('Invalid integer value')
    
    if isinstance(val, str):
        if val.casefold() in ['1', 'true', 't']:
            return True 
        elif val.casefold() in ['0', 'false', 'f']:
            return False
        else:
            raise ConversionError('Invalid string value')
    
    else:
        raise ConversionError("Invlid type, not implemented yet")

In [33]:
values = [True, 0, 'T', 'false', 10, 'ABC', 1.0]

for value in values:
    try:
        result = make_bool(val=value)
    except ConversionError as ex:
        result = str(ex)

    print(value, result)

True True
0 False
T True
false False
10 Invalid integer value
ABC Invalid string value
1.0 Invlid type, not implemented yet


Forgiveness approach

In [34]:
def get_item_forgive_me(seq, index, default=None):
    try:
        return seq[index]
    except IndexError as ex:
        return default 
    except TypeError as ex:
        return default
    except KeyError as ex:
        return default

Since they're all doing the same thing: 

In [35]:
def get_item_forgive_me(seq, index, default=None):
    try:
        return seq[index]
    except (IndexError, TypeError, KeyError) as ex:
        return default

Permission approach

In [36]:
def get_item_ask_perm(seq, index, default=None):
    if hasattr(seq, '__getitem__'):
        if index < len(seq):
            return seq[index]
    return default

In [37]:
get_item_forgive_me([1, 2, 3,], 0)

1

In [38]:
get_item_forgive_me([1, 2, 3,], 10, 'Nope')

'Nope'

In [39]:
get_item_ask_perm([1, 2, 3,], 0)

1

In [40]:
get_item_ask_perm([1, 2, 3,], 10, 'Nope')


'Nope'

In [41]:
get_item_forgive_me({'a':100}, 'a')

100

In [42]:
get_item_ask_perm({'a':100}, 'a')

TypeError: '<' not supported between instances of 'str' and 'int'